In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re,string,unicodedata
import nltk
from wordcloud import WordCloud
from nltk import ngrams
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
import spacy
nlp = spacy.load("en_core_web_sm")
from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import SVC
# from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.metrics import accuracy_score ,roc_auc_score,f1_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import GradientBoostingClassifier

import mlflow
import mlflow.sklearn

from lightgbm import LGBMClassifier

import streamlit as st


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Praveen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_train=pd.read_csv('train.csv',encoding='latin-1')

In [3]:
df_train.head()
#first 5 rows

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [4]:
df_train=df_train[['text','sentiment']]

In [5]:
df_train['text'] = df_train['text'].astype(str)

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       27481 non-null  object
 1   sentiment  27481 non-null  object
dtypes: object(2)
memory usage: 429.5+ KB


In [7]:
df_train=df_train[df_train['sentiment']!='neutral']
df_train['sentiment']=df_train['sentiment'].map({'positive':1,'negative':0})

In [8]:
X=df_train['text']
y=df_train['sentiment']

In [9]:
class TextCleaner():
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [self.clean_text(text) for text in X]
    def clean_text(self, text):
        text = str(text).lower()  # Make text lowercase
        text = re.sub('\[.*?\]', '', text)  # Remove any sequence of characters in square brackets
        text = re.sub('https?://\S+|www\.\S+', '', text)  # Remove links
        text = re.sub('<.*?>+', '', text)  # Remove HTML tags
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  # Remove punctuation
        text = re.sub('\n', '', text)  # Remove newline characters
        text = re.sub('\w*\d\w*', '', text)  # Remove words containing numbers
        text = re.sub(r'[^a-z/A-Z/0-9/ ]', '', text)  # Remove special characters
        return text

In [10]:
class stopwords():
    
    def fit(self,X,y=None):
        return self
    
    def transform(self, X):
        return [self.stopw(text) for text in X]
    
    def stopw(self,text):
        from nltk.corpus import stopwords
        stopwords = stopwords.words('english')
        stopwords=stopwords+['s','m','u','im','ye','id','atg','na','ta','gon','wan']
        text= ' '.join([x for x in text.split() if x not in stopwords])
        return text

In [11]:
class lemma():
    
    def __init__(self,lemma_model):
        self.lemma_model=lemma_model
        
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        return [self.lemmatise(text) for text in X]
    
    def lemmatise(self,text):
        return " ".join([token.lemma_ for token in self.lemma_model(text)])

In [12]:
def to_array(X):
    return X.toarray()
    

In [13]:
pipeline = Pipeline([
    ('cleaner', TextCleaner()),
    ('stop_words_removal',stopwords()),
    ('lemmatization',lemma(spacy.load("en_core_web_sm", disable = ['parser','ner']))),
    ('TF-IDF-fit',TfidfVectorizer()),
    #('TF-IDF-Transform',FunctionTransformer(to_array,accept_sparse=True))
    #('toarray',FunctionTransformer(to_array,accept_sparse=True)),
    ('model',MultinomialNB())
    
        
])

In [14]:
pipeline.fit(X,y)

Pipeline(steps=[('cleaner',
                 <__main__.TextCleaner object at 0x0000022D72CE4C10>),
                ('stop_words_removal',
                 <__main__.stopwords object at 0x0000022D72CE4C40>),
                ('lemmatization',
                 <__main__.lemma object at 0x0000022D72CE4FD0>),
                ('TF-IDF-fit', TfidfVectorizer()), ('model', MultinomialNB())])

In [15]:
pred=pipeline.predict(X)

In [16]:
accuracy = accuracy_score(y,pred)
accuracy

0.9306973048951903

In [17]:
f1 = f1_score(y,pred)
f1

0.9348200942637085

In [18]:
# Set the tracking URI to the local tracking server
mlflow.set_tracking_uri('http://127.0.0.1:5000')

In [19]:
# Define the experiment name
experiment_name = "mlflow_pipeline_experiment"

# Check if the experiment exists
experiment = mlflow.get_experiment_by_name(experiment_name)

In [20]:
if experiment is None:
    # If the experiment does not exist, create it
    mlflow.create_experiment(experiment_name)

In [21]:
# Set the experiment
mlflow.set_experiment(experiment_name)

# Log model with MLflow
with mlflow.start_run(run_name="Multinomial NB"):
    mlflow.sklearn.log_model(pipeline, "model")
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("f1 score", accuracy)

2024/01/15 14:42:45 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



In [22]:
algo_list = [XGBClassifier(),GradientBoostingClassifier(),MultinomialNB(),LGBMClassifier()]

In [23]:
# for x in algo_list:
#     pipeline = Pipeline([
#         ('cleaner', TextCleaner()),
#         ('stop_words_removal',stopwords()),
#         ('lemmatization',lemma(spacy.load("en_core_web_sm", disable = ['parser','ner']))),
#         ('TF-IDF-fit',TfidfVectorizer()),
#         #('TF-IDF-Transform',FunctionTransformer(to_array,accept_sparse=True))
#         #('toarray',FunctionTransformer(to_array,accept_sparse=True)),
#         ('model',x) 
#     ])

#     pipeline.fit(X,y)
#     accuracy = accuracy_score(y,pred)
#     f1 = f1_score(y,pred)
    
#     with mlflow.start_run():
#         mlflow.sklearn.log_model(pipeline, "model")
#         mlflow.log_metric("accuracy", accuracy)
#         mlflow.log_metric("f1 score", accuracy)

In [26]:
from sklearn.model_selection import train_test_split

for x in algo_list:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    pipeline = Pipeline([
        ('cleaner', TextCleaner()),
        ('stop_words_removal', stopwords()),
        ('lemmatization', lemma(spacy.load("en_core_web_sm", disable=['parser', 'ner']))),
        ('TF-IDF', TfidfVectorizer()),
        ('model', x)
    ])

    # Fit and transform TF-IDF on the training set
    X_train_tfidf = pipeline.named_steps['TF-IDF'].fit_transform(X_train)
    
    # Transform TF-IDF on the test set
    X_test_tfidf = pipeline.named_steps['TF-IDF'].transform(X_test)
    
    # Fit the rest of the pipeline on the training set
    pipeline.fit(X_train, y_train)

    y_pred_train = pipeline.predict(X_train)
    
    # Predict on the test set
    y_pred = pipeline.predict(X_test)
    
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred)

    f1_train = f1_score(y_train, y_pred_train)
    f1_test = f1_score(y_test, y_pred)

    with mlflow.start_run():
        mlflow.log_param("model_type", x)
        mlflow.sklearn.log_model(pipeline, "model")
        mlflow.log_metric("accuracy train", accuracy_train)
        mlflow.log_metric("accuracy test", accuracy_test)
        mlflow.log_metric("f1 train", f1_train)
        mlflow.log_metric("f1 test", f1_test)

  (0, 15640)	0.2571424906013482
  (0, 5412)	0.3149185950804328
  (0, 7661)	0.15086296860729959
  (0, 5635)	0.4775029992055459
  (0, 10402)	0.15758568958029026
  (0, 2313)	0.4775029992055459
  (0, 14689)	0.2981295086824993
  (0, 1713)	0.19643866155469816
  (0, 16294)	0.13561132285904076
  (0, 14343)	0.15665890842767735
  (0, 14327)	0.271267984604292
  (0, 10619)	0.29513520459704845
  (1, 15689)	0.267992502645706
  (1, 1427)	0.19426305023805265
  (1, 16033)	0.29206491192518536
  (1, 10277)	0.1940793020173602
  (1, 16300)	0.21493779607552865
  (1, 7559)	0.24517549121339025
  (1, 1928)	0.262466231735059
  (1, 7068)	0.4725838794885963
  (1, 7911)	0.2332528735518853
  (1, 8414)	0.23633050767743236
  (1, 856)	0.5086067853282885
  (2, 15686)	0.4594803810367507
  (2, 7803)	0.5761583924333249
  :	:
  (13088, 15970)	0.30304172092993864
  (13088, 9514)	0.3629418484321256
  (13088, 16049)	0.3245944418348514
  (13088, 15731)	0.2717365951616619
  (13088, 15223)	0.18264209065177958
  (13088, 5677)	0.4

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('cleaner', TextCleaner()),
    ('stop_words_removal', stopwords()),
    ('lemmatization', lemma(spacy.load("en_core_web_sm", disable=['parser', 'ner']))),
    ('TF-IDF', TfidfVectorizer()),
    ('model', LGBMClassifier())
])

# Fit and transform TF-IDF on the training set
X_train_tfidf = pipeline.named_steps['TF-IDF'].fit_transform(X_train)

# Transform TF-IDF on the test set
X_test_tfidf = pipeline.named_steps['TF-IDF'].transform(X_test)

# Fit the rest of the pipeline on the training set
pipeline.fit(X_train, y_train)

y_pred_train = pipeline.predict(X_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

[LightGBM] [Info] Number of positive: 6912, number of negative: 6178
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011562 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 21239
[LightGBM] [Info] Number of data points in the train set: 13090, number of used features: 660
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.528037 -> initscore=0.112264
[LightGBM] [Info] Start training from score 0.112264


In [34]:
# Create the Streamlit web app
st.header("Streamlit demo")

st.sidebar.header("This is a web app")

# X_test = st.sidebar.slider("Select X to get yhat", 0, 10, 5)

X_test_values = X_test  # You can replace this with your actual values

X_test = st.sidebar.selectbox("Select the sentence to know the sentiment", X_test_values)

st.write("X test is:", X_test)

if pipeline is not None:
    yhat_test = pipeline.predict([[X_test]])

    st.write("b0 is", yhat_test)

else:
    st.write("Model not found")